# Stage 2 Analysis

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from os import *
import sys
sys.path.append("../")
import urllib.request
import requests
import json

In [2]:
# Change directory
def change_dir(newpath):
    get_path = sys.path[0].split("\\")      
    del get_path[-1]                                                  
    get_path.append(newpath)                    
    path = "\\".join(get_path)              
    os.chdir(path)

## Demographics

In [3]:
# Get demographics data
api_query = "https://api.census.gov/data/2019/pep/charagegroups?get=POP,NAME,RACE,HISP,STATE&for=county&DATE_CODE=12"
response = requests.get(api_query)
formattedResponse = json.loads(response.text)
demographics = pd.DataFrame(formattedResponse)
demographics.columns = demographics.iloc[0]
demographics = demographics.iloc[1:,:]
demographics[['POP','RACE','HISP','STATE','state','county']] = demographics[['POP','RACE','HISP','STATE','state','county']].astype(int)
demographics = demographics[demographics['state']<57]
demographics = demographics[demographics['RACE']<7]
demographics

,POP,NAME,RACE,HISP,STATE,DATE_CODE,state,county
1,22124,"Coahoma County, Mississippi",0,0,28,12,28,27
2,21757,"Coahoma County, Mississippi",0,1,28,12,28,27
3,367,"Coahoma County, Mississippi",0,2,28,12,28,27
4,4626,"Coahoma County, Mississippi",1,0,28,12,28,27
5,4412,"Coahoma County, Mississippi",1,1,28,12,28,27
...,...,...,...,...,...,...,...,...
113171,0,"Stanton County, Kansas",5,1,20,12,20,187
113172,4,"Stanton County, Kansas",5,2,20,12,20,187
113173,72,"Stanton County, Kansas",6,0,20,12,20,187
113174,61,"Stanton County, Kansas",6,1,20,12,20,187


In [4]:
# Clean up races and ethinicity and parse as columns
demographics['RACE']= demographics['RACE'].replace({5: 4, 6: 3})
demographics['race_hisp'] = demographics['RACE'].astype(str) + '-' + demographics['HISP'].astype(str)
race_combinations = ['0-0','0-2','1-1','2-1','4-1','3-1']
demographics = demographics[demographics['race_hisp'].isin(race_combinations)]
demographics = pd.pivot_table(demographics, values='POP', index=['NAME','state','county'],columns=['race_hisp'],aggfunc=np.sum)
demographics = demographics.reset_index()
demographics = demographics.rename(columns={'0-0':'total_pop','0-2':'hispanic','1-1':'white_nonhispanic','2-1':'black_nonhispanic','4-1':'asian_nonhispanic','3-1':'other_nonhispanic'})
demographics

race_hisp,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic
0,"Abbeville County, South Carolina",45,1,24527,402,16892,6726,416,91
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212
2,"Accomack County, Virginia",51,1,32316,2955,19352,9104,626,279
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751
4,"Adair County, Iowa",19,1,7152,182,6803,41,86,40
...,...,...,...,...,...,...,...,...,...
3137,"Yuma County, Arizona",4,27,213787,138131,64453,3975,4495,2733
3138,"Yuma County, Colorado",8,125,10019,2518,7303,44,122,32
3139,"Zapata County, Texas",48,505,14179,13429,649,42,34,25
3140,"Zavala County, Texas",48,507,11840,11135,577,68,39,21


In [5]:
# Get age group data
api_query = "https://api.census.gov/data/2019/pep/charagegroups?get=POP,NAME,AGEGROUP&for=county&DATE_CODE=12"
response = requests.get(api_query)
formattedResponse = json.loads(response.text)
age = pd.DataFrame(formattedResponse)
age.columns = age.iloc[0]
age = age.iloc[1:,:]
age['POP'] = age['POP'].astype(float)
age[['state','county']] = age[['state','county']].astype(int)
age = age[age['state']<57]
age = age[age['AGEGROUP']=='26']
age

,POP,NAME,AGEGROUP,DATE_CODE,state,county
31,3399.0,"Jasper County, Mississippi",26,12,28,61
124,1231.0,"Quitman County, Mississippi",26,12,28,119
156,10601.0,"Pearl River County, Mississippi",26,12,28,109
193,4863.0,"Tate County, Mississippi",26,12,28,137
266,2291.0,"Lawrence County, Mississippi",26,12,28,77
...,...,...,...,...,...,...
102919,1152.0,"Sherman County, Kansas",26,12,20,181
102969,28886.0,"Scott County, Iowa",26,12,19,163
102990,11351.0,"Dallas County, Iowa",26,12,19,49
103013,2840.0,"O'Brien County, Iowa",26,12,19,141


In [6]:
# Merge age group with the demographics data
demographics = demographics.merge(age, on=['NAME','state','county'])
demographics = demographics.rename(columns={'POP':'above_65'})
demographics = demographics.drop(columns=['AGEGROUP','DATE_CODE'])
demographics

,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65
0,"Abbeville County, South Carolina",45,1,24527,402,16892,6726,416,91,5483.0
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212,9796.0
2,"Accomack County, Virginia",51,1,32316,2955,19352,9104,626,279,7934.0
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751,71958.0
4,"Adair County, Iowa",19,1,7152,182,6803,41,86,40,1648.0
...,...,...,...,...,...,...,...,...,...,...
3137,"Yuma County, Arizona",4,27,213787,138131,64453,3975,4495,2733,41302.0
3138,"Yuma County, Colorado",8,125,10019,2518,7303,44,122,32,1872.0
3139,"Zapata County, Texas",48,505,14179,13429,649,42,34,25,1866.0
3140,"Zavala County, Texas",48,507,11840,11135,577,68,39,21,1723.0


In [7]:
# Get education data - ACS 5 year estimates
api_query = "https://api.census.gov/data/2019/acs/acs5/cprofile?get=NAME,CP02_2019_060E,CP02_2019_061E,CP02_2019_062E,CP02_2019_063E,CP02_2019_064E,CP02_2019_068E&for=county"
response = requests.get(api_query)
formattedResponse = json.loads(response.text)
education = pd.DataFrame(formattedResponse)
education.columns = education.iloc[0]
education = education.iloc[1:,:]
education = education.rename(columns={'CP02_2019_060E':'less_than_9grade','CP02_2019_061E':'hs_nodiploma','CP02_2019_062E':'ged','CP02_2019_063E':'college_nodegree','CP02_2019_064E':'associates','CP02_2019_068E':'bachelors_graduate'})
education[['less_than_9grade','hs_nodiploma','ged','college_nodegree','associates','bachelors_graduate']] = education[['less_than_9grade','hs_nodiploma','ged','college_nodegree','associates','bachelors_graduate']].astype(float)
education[['state','county']] = education[['state','county']].astype(int)
education = education[education['state']<57]
education

,NAME,less_than_9grade,hs_nodiploma,ged,college_nodegree,associates,bachelors_graduate,state,county
1,"Fayette County, Illinois",4.9,11.3,40.2,22.4,10.0,11.3,17,51
2,"Logan County, Illinois",3.2,8.1,35.3,24.1,9.4,19.9,17,107
3,"Saline County, Illinois",4.4,8.9,27.5,26.4,13.6,19.2,17,165
4,"Lake County, Illinois",4.9,4.4,20.8,18.2,6.3,45.3,17,97
5,"Massac County, Illinois",5.0,8.2,33.6,27.1,12.1,13.9,17,127
...,...,...,...,...,...,...,...,...,...
2900,"Crockett County, Tennessee",6.1,12.7,40.5,20.0,6.8,13.8,47,33
2901,"Lake County, Tennessee",9.9,17.1,43.2,16.5,3.8,9.4,47,95
2902,"Knox County, Tennessee",2.6,5.7,25.3,20.5,8.3,37.6,47,93
2903,"Benton County, Washington",5.2,4.7,24.5,24.6,10.2,30.9,53,5


In [8]:
# Merge education with the demographics dataset
merged_demographics = pd.merge(demographics,education,on=['NAME','county','state'],how='left').drop_duplicates()
merged_demographics

,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65,less_than_9grade,hs_nodiploma,ged,college_nodegree,associates,bachelors_graduate
0,"Abbeville County, South Carolina",45,1,24527,402,16892,6726,416,91,5483.0,6.0,12.4,34.0,18.9,13.1,15.6
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212,9796.0,8.9,12.1,41.3,17.8,6.6,13.3
2,"Accomack County, Virginia",51,1,32316,2955,19352,9104,626,279,7934.0,6.6,11.9,36.7,17.1,8.2,19.5
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751,71958.0,1.4,3.4,22.4,24.3,9.9,38.5
4,"Adair County, Iowa",19,1,7152,182,6803,41,86,40,1648.0,2.1,3.7,40.5,20.6,14.6,18.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,"Yuma County, Arizona",4,27,213787,138131,64453,3975,4495,2733,41302.0,13.5,13.2,25.7,24.8,7.7,15.0
3138,"Yuma County, Colorado",8,125,10019,2518,7303,44,122,32,1872.0,6.5,4.9,34.4,24.5,8.0,21.8
3139,"Zapata County, Texas",48,505,14179,13429,649,42,34,25,1866.0,24.0,14.1,30.8,15.6,3.9,11.6
3140,"Zavala County, Texas",48,507,11840,11135,577,68,39,21,1723.0,19.9,13.2,32.7,17.5,5.8,10.9


In [9]:
# Export as a csv
change_dir('output//stage2')
merged_demographics.to_csv('demographics.csv')

## Industry

In [10]:
# Retrieve Industry Data
change_dir('output')
industry = pd.read_csv('merged_industry_LAUS.csv')
NECTA = [16980,19100,19820,31080,33100,35620,37980,41860,42660,47900,71650]
industry = industry[~industry['area_code'].isin(NECTA)]
industry

,year,period,industry_employment,state_code,area_code,supersector_code,industry_code,unemployment_rate,unemployment,employment,labor_force
0,2015,M01,45.8,1,11500,0,0,7.6,3506.0,42824.0,46330.0
1,2015,M01,33.5,1,11500,5,5000000,7.6,3506.0,42824.0,46330.0
2,2015,M01,6.7,1,11500,6,6000000,7.6,3506.0,42824.0,46330.0
3,2015,M01,39.1,1,11500,7,7000000,7.6,3506.0,42824.0,46330.0
4,2015,M01,26.8,1,11500,8,8000000,7.6,3506.0,42824.0,46330.0
...,...,...,...,...,...,...,...,...,...,...,...
913411,2021,M06,1.6,56,16940,80,80000000,5.0,2567.0,48663.0,51230.0
913412,2021,M06,13.7,56,16940,90,90000000,5.0,2567.0,48663.0,51230.0
913413,2021,M06,2.8,56,16940,90,90910000,5.0,2567.0,48663.0,51230.0
913414,2021,M06,3.9,56,16940,90,90920000,5.0,2567.0,48663.0,51230.0


In [11]:
industry[(industry['industry_code']==0) & (industry['year']==2020) & (industry['period']=='M02')]

,year,period,industry_employment,state_code,area_code,supersector_code,industry_code,unemployment_rate,unemployment,employment,labor_force
1320,2020,M02,47.5,1,11500,0,0,3.3,1509.0,44586.0,46095.0
3000,2020,M02,67.4,1,12220,0,0,2.4,1820.0,74728.0,76548.0
6132,2020,M02,544.6,1,13820,0,0,2.5,13581.0,536973.0,550554.0
7284,2020,M02,79.7,1,19300,0,0,2.5,2447.0,94213.0,96660.0
8844,2020,M02,57.9,1,19460,0,0,2.4,1748.0,71243.0,72991.0
...,...,...,...,...,...,...,...,...,...,...,...
905460,2020,M02,76.6,55,39540,0,0,4.3,4211.0,92649.0,96860.0
907506,2020,M02,61.6,55,43100,0,0,2.8,1683.0,58807.0,60490.0
909342,2020,M02,72.3,55,48140,0,0,2.9,2089.0,70012.0,72101.0
911238,2020,M02,39.0,56,16220,0,0,5.2,2123.0,38769.0,40892.0


In [12]:
# Drop columns and rows that do not correspond to the total, manufacturing and service counts
industry_codes = [0,6000000,7000000]
industry1 = industry[industry['industry_code'].isin(industry_codes)]
industry1 = industry1.drop(columns=['supersector_code','unemployment','employment','unemployment_rate','labor_force'])
industry1

,year,period,industry_employment,state_code,area_code,industry_code
0,2015,M01,45.8,1,11500,0
2,2015,M01,6.7,1,11500,6000000
3,2015,M01,39.1,1,11500,7000000
20,2015,M02,46.0,1,11500,0
22,2015,M02,6.7,1,11500,6000000
...,...,...,...,...,...,...
913376,2021,M05,4.9,56,16940,6000000
913377,2021,M05,42.1,56,16940,7000000
913395,2021,M06,47.0,56,16940,0
913397,2021,M06,4.9,56,16940,6000000


In [13]:
# Filter for total 2019, feb 2020, and june 2021 data
industry1 = industry1[((industry1['year']==2019) & (industry1['period']=='M02')) | ((industry1['year']==2020) & (industry1['period']=='M06')) | ((industry1['year']==2019) & (industry1['period']=='M13'))]
industry1

,year,period,industry_employment,state_code,area_code,industry_code
1060,2019,M02,46.9,1,11500,0
1062,2019,M02,7.8,1,11500,6000000
1063,2019,M02,39.1,1,11500,7000000
1280,2019,M13,47.3,1,11500,0
1282,2019,M13,7.8,1,11500,6000000
...,...,...,...,...,...,...
912998,2019,M13,5.5,56,16940,6000000
912999,2019,M13,42.2,56,16940,7000000
913122,2020,M06,45.8,56,16940,0
913124,2020,M06,5.1,56,16940,6000000


In [14]:
# Parse industries as columns
industry1 = pd.pivot_table(industry1, values='industry_employment', index=['year','period','state_code','area_code'],columns=['industry_code'],aggfunc=np.sum).reset_index()
industry1 = industry1.rename(columns={0:'total_employment',6000000:'manufacturing',7000000:'service'})
industry1

industry_code,year,period,state_code,area_code,total_employment,manufacturing,service
0,2019,M02,1,11500,46.9,7.8,39.1
1,2019,M02,1,12220,66.3,9.0,57.3
2,2019,M02,1,13820,541.3,71.0,470.3
3,2019,M02,1,19300,78.1,9.2,68.9
4,2019,M02,1,19460,56.6,17.1,39.5
...,...,...,...,...,...,...,...
1243,2020,M06,55,39540,73.5,20.0,53.5
1244,2020,M06,55,43100,59.7,22.6,37.1
1245,2020,M06,55,48140,69.5,20.9,48.6
1246,2020,M06,56,16220,35.8,5.8,30.0


In [15]:
# Split data for each time period into columns and merge 
industry1['month'] = industry1['period'].astype(str) + '-' + industry1['year'].astype(str)
industry1 = industry1.drop(columns=['year','period'])
feb2020 = industry1[industry1['month'] == 'M02-2019']
jun2021 = industry1[industry1['month'] == 'M06-2020']
annual2019 = industry1[industry1['month'] == 'M13-2019']
feb2020 = feb2020.rename(columns={'total_employment':'total_emp_feb2020','manufacturing':'manufacturing_emp_feb2020','service':'service_emp_feb2020'})
jun2021 = jun2021.rename(columns={'total_employment':'total_emp_jun2021','manufacturing':'manufacturing_emp_jun2021','service':'service_emp_jun2021'})
merged = pd.merge(pd.merge(feb2020,jun2021,on=['state_code','area_code']),annual2019,on=['state_code','area_code'])
merged = merged.drop(columns=['month','month_x','month_y'])
merged

industry_code,state_code,area_code,total_emp_feb2020,manufacturing_emp_feb2020,service_emp_feb2020,total_emp_jun2021,manufacturing_emp_jun2021,service_emp_jun2021,total_employment,manufacturing,service
0,1,11500,46.9,7.8,39.1,44.4,7.6,36.8,47.3,7.8,39.5
1,1,12220,66.3,9.0,57.3,60.8,8.7,52.1,67.3,9.0,58.3
2,1,13820,541.3,71.0,470.3,507.7,67.6,440.1,546.1,71.8,474.4
3,1,19300,78.1,9.2,68.9,77.8,8.6,69.2,80.5,9.1,71.4
4,1,19460,56.6,17.1,39.5,55.6,17.6,38.0,57.5,17.5,40.0
...,...,...,...,...,...,...,...,...,...,...,...
411,55,39540,77.4,20.7,56.7,73.5,20.0,53.5,78.7,20.8,57.8
412,55,43100,62.2,24.9,37.3,59.7,22.6,37.1,63.5,25.0,38.5
413,55,48140,72.3,20.7,51.6,69.5,20.9,48.6,73.5,21.3,52.1
414,56,16220,38.9,7.3,31.6,35.8,5.8,30.0,39.6,7.5,32.1


In [16]:
# Add columns computing the change
merged['emp_tot_change_feb2020_jun2021'] = (merged['total_emp_jun2021']/merged['total_emp_feb2020'])-1
merged['manufacturing_change_feb2020_jun2021'] = (merged['manufacturing_emp_jun2021']/merged['manufacturing_emp_feb2020'])-1
merged['service_change_feb2020_jun2021'] = (merged['service_emp_jun2021']/merged['service_emp_feb2020'])-1
merged

industry_code,state_code,area_code,total_emp_feb2020,manufacturing_emp_feb2020,service_emp_feb2020,total_emp_jun2021,manufacturing_emp_jun2021,service_emp_jun2021,total_employment,manufacturing,service,emp_tot_change_feb2020_jun2021,manufacturing_change_feb2020_jun2021,service_change_feb2020_jun2021
0,1,11500,46.9,7.8,39.1,44.4,7.6,36.8,47.3,7.8,39.5,-0.053305,-0.025641,-0.058824
1,1,12220,66.3,9.0,57.3,60.8,8.7,52.1,67.3,9.0,58.3,-0.082956,-0.033333,-0.090750
2,1,13820,541.3,71.0,470.3,507.7,67.6,440.1,546.1,71.8,474.4,-0.062073,-0.047887,-0.064214
3,1,19300,78.1,9.2,68.9,77.8,8.6,69.2,80.5,9.1,71.4,-0.003841,-0.065217,0.004354
4,1,19460,56.6,17.1,39.5,55.6,17.6,38.0,57.5,17.5,40.0,-0.017668,0.029240,-0.037975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,55,39540,77.4,20.7,56.7,73.5,20.0,53.5,78.7,20.8,57.8,-0.050388,-0.033816,-0.056437
412,55,43100,62.2,24.9,37.3,59.7,22.6,37.1,63.5,25.0,38.5,-0.040193,-0.092369,-0.005362
413,55,48140,72.3,20.7,51.6,69.5,20.9,48.6,73.5,21.3,52.1,-0.038728,0.009662,-0.058140
414,56,16220,38.9,7.3,31.6,35.8,5.8,30.0,39.6,7.5,32.1,-0.079692,-0.205479,-0.050633


In [17]:
# Compute increase in total employment since 2015
industry2 = industry[industry['period']=='M13']
industry2 = industry2[(industry2['year']==2015) | (industry2['year']==2019)]
industry2 = industry2[industry2['industry_code']==0]
industry2 = industry2.drop(columns=['supersector_code','unemployment','employment','unemployment_rate','labor_force','industry_code','period'])
year_15 = industry2.loc[industry2['year'] == 2015][['industry_employment','area_code']]
year_15 = year_15.drop_duplicates()
year_19 = industry2.loc[industry2['year'] == 2019][['industry_employment','area_code']]
year_19 = year_19.drop_duplicates()
prepandemic = year_15.merge(year_19, on='area_code')
prepandemic = prepandemic.rename(columns={'industry_employment_x':'emp_15','industry_employment_y':'emp_19'})
prepandemic['prepandemic_growth'] = (prepandemic['emp_19'] - prepandemic['emp_15'])/prepandemic['emp_15']
prepandemic = prepandemic.drop(columns=['emp_15','emp_19'])
industry2 = industry2[industry2['year']==2019]
industry2 = industry2.merge(prepandemic, on='area_code')
industry2 = industry2.drop(columns=['year','industry_employment'])
industry2

,state_code,area_code,prepandemic_growth
0,1,11500,0.023810
1,1,12220,0.114238
2,1,13820,0.046369
3,1,19300,0.146724
4,1,19460,0.066790
...,...,...,...
411,55,39540,0.022078
412,55,43100,0.042693
413,55,48140,0.019417
414,56,16220,-0.063830


In [18]:
# Compute employment_leisure/total_employment
leisure = industry.loc[(industry['industry_code'] == 70000000) | (industry['industry_code'] == 0)][['industry_employment','area_code','industry_code','state_code']]
leisure = pd.pivot_table(leisure, values='industry_employment', index=['area_code','state_code'],columns=['industry_code'],aggfunc=np.sum).reset_index()
leisure = leisure.rename(columns={0:'total_employment',70000000:'leisure'})
leisure['leisure_by_emp'] = leisure['leisure']/leisure['total_employment']
leisure = leisure.drop(columns=['total_employment','leisure'])
leisure

industry_code,area_code,state_code,leisure_by_emp
0,10180,48,0.111297
1,10420,39,0.096521
2,10500,13,NaN
3,10540,41,0.079524
4,10580,36,0.082655
...,...,...,...
411,77200,44,0.111436
412,78100,25,0.089903
413,78254,25,0.092125
414,78700,9,0.081721


In [19]:
# Merge additional columns
merged_all = pd.merge(pd.merge(industry2,leisure,on=['state_code','area_code']),merged,on=['state_code','area_code'])
merged_all

,state_code,area_code,prepandemic_growth,leisure_by_emp,total_emp_feb2020,manufacturing_emp_feb2020,service_emp_feb2020,total_emp_jun2021,manufacturing_emp_jun2021,service_emp_jun2021,total_employment,manufacturing,service,emp_tot_change_feb2020_jun2021,manufacturing_change_feb2020_jun2021,service_change_feb2020_jun2021
0,1,11500,0.023810,0.105036,46.9,7.8,39.1,44.4,7.6,36.8,47.3,7.8,39.5,-0.053305,-0.025641,-0.058824
1,1,12220,0.114238,0.128200,66.3,9.0,57.3,60.8,8.7,52.1,67.3,9.0,58.3,-0.082956,-0.033333,-0.090750
2,1,13820,0.046369,0.094002,541.3,71.0,470.3,507.7,67.6,440.1,546.1,71.8,474.4,-0.062073,-0.047887,-0.064214
3,1,19300,0.146724,NaN,78.1,9.2,68.9,77.8,8.6,69.2,80.5,9.1,71.4,-0.003841,-0.065217,0.004354
4,1,19460,0.066790,0.092045,56.6,17.1,39.5,55.6,17.6,38.0,57.5,17.5,40.0,-0.017668,0.029240,-0.037975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,55,39540,0.022078,0.097312,77.4,20.7,56.7,73.5,20.0,53.5,78.7,20.8,57.8,-0.050388,-0.033816,-0.056437
412,55,43100,0.042693,0.083516,62.2,24.9,37.3,59.7,22.6,37.1,63.5,25.0,38.5,-0.040193,-0.092369,-0.005362
413,55,48140,0.019417,0.078955,72.3,20.7,51.6,69.5,20.9,48.6,73.5,21.3,52.1,-0.038728,0.009662,-0.058140
414,56,16220,-0.063830,0.116439,38.9,7.3,31.6,35.8,5.8,30.0,39.6,7.5,32.1,-0.079692,-0.205479,-0.050633


In [20]:
change_dir('output//stage2')
merged_all.to_csv('industries.csv')

## Merging 

In [21]:
# Merge the CBSA crosswalk
change_dir('input')
add_CBSA_codes = pd.read_csv('delineation_files.csv')
add_CBSA_codes = add_CBSA_codes.rename(columns={"CBSA Code":'area_code',"FIPS State Code": "state", "FIPS County Code": "county"})
add_CBSA_codes[['state','county']] = add_CBSA_codes[['state','county']].astype(int)
add_CBSA_codes = add_CBSA_codes.drop(columns=['State Name','County/County Equivalent'])
demographics_CBSA = merged_demographics.merge(add_CBSA_codes,left_on=['county','state'],right_on=['county','state'],how='left').drop_duplicates()
demographics_CBSA

,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65,less_than_9grade,hs_nodiploma,ged,college_nodegree,associates,bachelors_graduate,area_code,CBSA Title
0,"Abbeville County, South Carolina",45,1,24527,402,16892,6726,416,91,5483.0,6.0,12.4,34.0,18.9,13.1,15.6,24940.0,"Greenwood, SC"
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212,9796.0,8.9,12.1,41.3,17.8,6.6,13.3,29180.0,"Lafayette, LA"
2,"Accomack County, Virginia",51,1,32316,2955,19352,9104,626,279,7934.0,6.6,11.9,36.7,17.1,8.2,19.5,NaN,NaN
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751,71958.0,1.4,3.4,22.4,24.3,9.9,38.5,14260.0,"Boise City, ID"
4,"Adair County, Iowa",19,1,7152,182,6803,41,86,40,1648.0,2.1,3.7,40.5,20.6,14.6,18.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,"Yuma County, Arizona",4,27,213787,138131,64453,3975,4495,2733,41302.0,13.5,13.2,25.7,24.8,7.7,15.0,49740.0,"Yuma, AZ"
4003,"Yuma County, Colorado",8,125,10019,2518,7303,44,122,32,1872.0,6.5,4.9,34.4,24.5,8.0,21.8,NaN,NaN
4004,"Zapata County, Texas",48,505,14179,13429,649,42,34,25,1866.0,24.0,14.1,30.8,15.6,3.9,11.6,49820.0,"Zapata, TX"
4005,"Zavala County, Texas",48,507,11840,11135,577,68,39,21,1723.0,19.9,13.2,32.7,17.5,5.8,10.9,NaN,NaN


In [22]:
# Merge the above crosswalk with industry data
merged_all = merged_all.rename(columns={'state_code':'state'})
demographics_industry_CBSA = demographics_CBSA.merge(merged_all,left_on=['area_code','state'],right_on=['area_code','state'],how='left').drop_duplicates()
demographics_industry_CBSA

,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65,...,service_emp_feb2020,total_emp_jun2021,manufacturing_emp_jun2021,service_emp_jun2021,total_employment,manufacturing,service,emp_tot_change_feb2020_jun2021,manufacturing_change_feb2020_jun2021,service_change_feb2020_jun2021
0,"Abbeville County, South Carolina",45,1,24527,402,16892,6726,416,91,5483.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212,9796.0,...,165.6,185.8,33.6,152.2,205.2,39.1,166.0,-0.088769,-0.122715,-0.080918
2,"Accomack County, Virginia",51,1,32316,2955,19352,9104,626,279,7934.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751,71958.0,...,283.4,340.4,57.7,282.7,345.3,55.8,289.4,0.008593,0.066543,-0.002470
4,"Adair County, Iowa",19,1,7152,182,6803,41,86,40,1648.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,"Yuma County, Arizona",4,27,213787,138131,64453,3975,4495,2733,41302.0,...,53.3,52.7,5.9,46.8,57.6,5.7,51.9,-0.108291,0.017241,-0.121951
3248,"Yuma County, Colorado",8,125,10019,2518,7303,44,122,32,1872.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3249,"Zapata County, Texas",48,505,14179,13429,649,42,34,25,1866.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3250,"Zavala County, Texas",48,507,11840,11135,577,68,39,21,1723.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Filter out micro areas
demographics_industry_CBSA = demographics_industry_CBSA[demographics_industry_CBSA['total_emp_feb2020'].notna()]
demographics_industry_CBSA

,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65,...,service_emp_feb2020,total_emp_jun2021,manufacturing_emp_jun2021,service_emp_jun2021,total_employment,manufacturing,service,emp_tot_change_feb2020_jun2021,manufacturing_change_feb2020_jun2021,service_change_feb2020_jun2021
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212,9796.0,...,165.6,185.8,33.6,152.2,205.2,39.1,166.0,-0.088769,-0.122715,-0.080918
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751,71958.0,...,283.4,340.4,57.7,282.7,345.3,55.8,289.4,0.008593,0.066543,-0.002470
8,"Adams County, Colorado",8,1,517421,211139,253680,16842,13468,22292,55528.0,...,1326.6,1428.4,177.6,1250.8,1538.4,182.7,1355.7,-0.050645,-0.002247,-0.057139
17,"Adams County, Pennsylvania",42,1,103009,7511,91400,1692,1550,856,21730.0,...,26.0,30.7,8.8,21.9,35.2,9.4,25.8,-0.130312,-0.053763,-0.157692
20,"Addison County, Vermont",50,1,36777,841,34046,449,763,678,7598.0,...,108.0,110.4,17.9,92.5,126.5,18.8,107.7,-0.125891,-0.021858,-0.143519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237,"York County, Maine",23,31,207641,3830,194994,2004,3971,2842,43919.0,...,183.2,190.9,24.0,166.9,213.1,24.6,188.5,-0.076886,0.016949,-0.088974
3241,"York County, Pennsylvania",42,133,449058,36265,370992,26005,9091,6705,80627.0,...,142.7,170.7,42.1,128.6,188.2,44.1,144.1,-0.079288,-0.014052,-0.098809
3243,"York County, Virginia",51,199,68280,4701,47869,8990,2510,4210,11367.0,...,689.0,733.5,95.1,638.4,797.5,97.5,700.0,-0.066200,-0.014508,-0.073440
3245,"Yuba County, California",6,115,78668,22925,42455,2841,4877,5570,10270.0,...,39.8,45.9,5.0,40.9,46.3,5.2,41.1,0.029148,0.041667,0.027638


In [24]:
# Filter out for education nans
demographics_df = demographics_industry_CBSA.drop(columns=['prepandemic_growth', 'leisure_by_emp','total_emp_feb2020', 'manufacturing_emp_feb2020', 'service_emp_feb2020',
       'total_emp_jun2021', 'manufacturing_emp_jun2021', 'service_emp_jun2021', 'total_employment', 'manufacturing', 'service',
       'emp_tot_change_feb2020_jun2021','manufacturing_change_feb2020_jun2021','service_change_feb2020_jun2021'])
demographics_df

,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65,less_than_9grade,hs_nodiploma,ged,college_nodegree,associates,bachelors_graduate,area_code,CBSA Title
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212,9796.0,8.9,12.1,41.3,17.8,6.6,13.3,29180.0,"Lafayette, LA"
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751,71958.0,1.4,3.4,22.4,24.3,9.9,38.5,14260.0,"Boise City, ID"
8,"Adams County, Colorado",8,1,517421,211139,253680,16842,13468,22292,55528.0,7.3,8.9,28.6,22.0,8.9,24.3,19740.0,"Denver-Aurora-Lakewood, CO"
17,"Adams County, Pennsylvania",42,1,103009,7511,91400,1692,1550,856,21730.0,4.0,7.2,39.9,17.9,8.7,22.3,23900.0,"Gettysburg, PA"
20,"Addison County, Vermont",50,1,36777,841,34046,449,763,678,7598.0,2.2,4.3,29.1,16.2,8.6,39.6,72400.0,"Burlington-South Burlington, VT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237,"York County, Maine",23,31,207641,3830,194994,2004,3971,2842,43919.0,2.4,4.4,30.0,20.3,10.5,32.5,76750.0,"Portland-South Portland, ME"
3241,"York County, Pennsylvania",42,133,449058,36265,370992,26005,9091,6705,80627.0,3.1,7.0,39.6,16.3,9.2,24.9,49620.0,"York-Hanover, PA"
3243,"York County, Virginia",51,199,68280,4701,47869,8990,2510,4210,11367.0,2.3,3.0,18.9,18.9,9.8,47.1,47260.0,"Virginia Beach-Norfolk-Newport News, VA-NC"
3245,"Yuba County, California",6,115,78668,22925,42455,2841,4877,5570,10270.0,7.0,10.7,23.6,30.9,10.8,17.1,49700.0,"Yuba City, CA"


In [25]:
# Add a column for population sums to use a base for proportion computation
demographics_df['pop_excl'] = demographics_df['bachelors_graduate'].isna()
demographics_df['pop_excl'] = demographics_df['pop_excl'].apply(lambda x: 0 if x==True else '')
demographics_df['pop_excl'] = np.where(demographics_df['pop_excl'] == '', demographics_df['total_pop'], demographics_df['pop_excl'])
demographics_df['pop_excl'] = demographics_df['pop_excl'].astype(int)
demographics_df

,NAME,state,county,total_pop,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65,less_than_9grade,hs_nodiploma,ged,college_nodegree,associates,bachelors_graduate,area_code,CBSA Title,pop_excl
1,"Acadia Parish, Louisiana",22,1,62045,1732,47971,11009,1121,212,9796.0,8.9,12.1,41.3,17.8,6.6,13.3,29180.0,"Lafayette, LA",62045
3,"Ada County, Idaho",16,1,481587,41045,405966,6144,14681,13751,71958.0,1.4,3.4,22.4,24.3,9.9,38.5,14260.0,"Boise City, ID",481587
8,"Adams County, Colorado",8,1,517421,211139,253680,16842,13468,22292,55528.0,7.3,8.9,28.6,22.0,8.9,24.3,19740.0,"Denver-Aurora-Lakewood, CO",517421
17,"Adams County, Pennsylvania",42,1,103009,7511,91400,1692,1550,856,21730.0,4.0,7.2,39.9,17.9,8.7,22.3,23900.0,"Gettysburg, PA",103009
20,"Addison County, Vermont",50,1,36777,841,34046,449,763,678,7598.0,2.2,4.3,29.1,16.2,8.6,39.6,72400.0,"Burlington-South Burlington, VT",36777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237,"York County, Maine",23,31,207641,3830,194994,2004,3971,2842,43919.0,2.4,4.4,30.0,20.3,10.5,32.5,76750.0,"Portland-South Portland, ME",207641
3241,"York County, Pennsylvania",42,133,449058,36265,370992,26005,9091,6705,80627.0,3.1,7.0,39.6,16.3,9.2,24.9,49620.0,"York-Hanover, PA",449058
3243,"York County, Virginia",51,199,68280,4701,47869,8990,2510,4210,11367.0,2.3,3.0,18.9,18.9,9.8,47.1,47260.0,"Virginia Beach-Norfolk-Newport News, VA-NC",68280
3245,"Yuba County, California",6,115,78668,22925,42455,2841,4877,5570,10270.0,7.0,10.7,23.6,30.9,10.8,17.1,49700.0,"Yuba City, CA",78668


In [26]:
# Compute missing education values
demographics_df_grouped = demographics_df[demographics_df['pop_excl'] != 0]
demographics_df_grouped = demographics_df_grouped.drop(columns=['hispanic','white_nonhispanic','black_nonhispanic', 'other_nonhispanic', 'asian_nonhispanic','above_65','state','county','NAME'])
demographics_df_grouped[['less_than_9grade','hs_nodiploma','ged','college_nodegree','associates','bachelors_graduate']] = demographics_df_grouped[['less_than_9grade','hs_nodiploma','ged','college_nodegree','associates','bachelors_graduate']].multiply(demographics_df_grouped["total_pop"],axis="index").round(4)/100
demographics_df_grouped = demographics_df_grouped.groupby(['CBSA Title','area_code'])[['total_pop', 'pop_excl','less_than_9grade','hs_nodiploma','ged','college_nodegree','associates','bachelors_graduate']].sum().reset_index()
demographics_df_grouped[['less_than_9grade','hs_nodiploma','ged','college_nodegree','associates','bachelors_graduate']] = demographics_df_grouped[['less_than_9grade','hs_nodiploma','ged','college_nodegree','associates','bachelors_graduate']].divide(demographics_df_grouped["pop_excl"],axis="index").round(4)*100
demographics_df_grouped

,CBSA Title,area_code,total_pop,pop_excl,less_than_9grade,hs_nodiploma,ged,college_nodegree,associates,bachelors_graduate
0,"Abilene, TX",10180.0,172060,172060,4.33,7.73,32.28,24.66,7.70,23.28
1,"Akron, OH",10420.0,703479,703479,2.24,5.90,31.68,20.30,8.27,31.69
2,"Albany, GA",10500.0,146726,146726,4.35,11.13,31.97,22.81,9.55,20.19
3,"Albany, OR",10540.0,129749,129749,2.90,7.60,29.00,31.20,10.00,19.30
4,"Albany-Schenectady-Troy, NY",10580.0,880381,880381,2.50,5.29,26.17,16.21,12.37,37.53
...,...,...,...,...,...,...,...,...,...,...
411,"Yakima, WA",49420.0,250873,250873,15.30,11.00,28.10,20.70,8.20,16.70
412,"York-Hanover, PA",49620.0,449058,449058,3.10,7.00,39.60,16.30,9.20,24.90
413,"Youngstown-Warren-Boardman, OH-PA",49660.0,426657,426657,2.72,7.08,40.80,20.15,7.51,21.65
414,"Yuba City, CA",49700.0,175639,175639,9.87,10.09,23.43,28.08,10.80,17.71


In [27]:
# Final merge
demographics_industry_CBSA = demographics_industry_CBSA.drop(columns=["total_pop","less_than_9grade","hs_nodiploma","ged","college_nodegree","associates","bachelors_graduate","state","county"])
demographics_industry_CBSA = demographics_industry_CBSA.groupby(['CBSA Title','area_code']).sum().reset_index()
merged = demographics_industry_CBSA.merge(demographics_df_grouped,on=['CBSA Title','area_code'])
merged['above_65'] = merged['above_65']/merged['total_pop']*100
merged

,CBSA Title,area_code,hispanic,white_nonhispanic,black_nonhispanic,other_nonhispanic,asian_nonhispanic,above_65,prepandemic_growth,leisure_by_emp,...,manufacturing_change_feb2020_jun2021,service_change_feb2020_jun2021,total_pop,pop_excl,less_than_9grade,hs_nodiploma,ged,college_nodegree,associates,bachelors_graduate
0,"Abilene, TX",10180.0,41494,110094,12888,4241,3343,15.271998,0.184751,0.333892,...,-0.182927,-0.090476,172060,172060,4.33,7.73,32.28,24.66,7.70,23.28
1,"Akron, OH",10420.0,15607,558111,87860,17456,24445,18.158182,0.016578,0.193042,...,-0.068053,-0.169958,703479,703479,2.24,5.90,31.68,20.30,8.27,31.69
2,"Albany, GA",10500.0,4574,60476,80749,2130,1835,16.949280,0.139116,0.000000,...,-0.357143,-0.226449,146726,146726,4.35,11.13,31.97,22.81,9.55,20.19
3,"Albany, OR",10540.0,12355,109436,775,5444,1739,19.069126,0.106132,0.079524,...,-0.067227,-0.072046,129749,129749,2.90,7.60,29.00,31.20,10.00,19.30
4,"Albany-Schenectady-Troy, NY",10580.0,47738,698303,72192,22290,39858,17.980738,0.167506,0.413273,...,0.243056,-0.585245,880381,880381,2.50,5.29,26.17,16.21,12.37,37.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,"Yakima, WA",49420.0,125816,106175,2101,13570,3211,14.036186,0.066095,0.089198,...,0.000000,-0.050964,250873,250873,15.30,11.00,28.10,20.70,8.20,16.70
412,"York-Hanover, PA",49620.0,36265,370992,26005,9091,6705,17.954696,0.040929,0.084918,...,-0.014052,-0.098809,449058,449058,3.10,7.00,39.60,16.30,9.20,24.90
413,"Youngstown-Warren-Boardman, OH-PA",49660.0,18603,344527,50989,9322,3216,21.725180,-0.080602,0.219292,...,-0.184874,-0.197105,426657,426657,2.72,7.08,40.80,20.15,7.51,21.65
414,"Yuba City, CA",49700.0,53832,86028,4949,9104,21726,14.441553,0.264059,0.196372,...,0.083333,0.055276,175639,175639,9.87,10.09,23.43,28.08,10.80,17.71


In [28]:
# Save the final output
change_dir('output\\stage2')
merged.to_csv('output.csv')